# Logistics Training Pipeline (Cleaned)

This notebook trains global and carrier-specific LightGBM models to predict shipment **Cost** (log target) and **Transit_Days**,
applies **per-carrier calibration**, evaluates performance, and writes Streamlit-friendly artifacts:

- `model_artifacts/preprocessor.pkl`  
- `model_artifacts/cost_model.pkl`  
- `model_artifacts/cost_models_grouped.pkl` (optional carrier-specific)  
- `model_artifacts/preprocessor_days.pkl`  
- `model_artifacts/transit_days_model.pkl`  
- `model_artifacts/best_params.json` (includes features + calibration)  
- `model_artifacts/carrier_performance.csv`  
- `model_artifacts/carrier_rmse_analysis.png`

You can configure data sources via environment variables:

- **MySQL (preferred)**: `MYSQL_USER`, `MYSQL_PASSWORD`, `MYSQL_HOST`, `MYSQL_PORT`, `MYSQL_DB`  
- **Fallback CSV**: `LOGISTICS_CSV_PATH` (expects columns similar to the sample dataset)

Run the final cell to execute `main()`.


In [8]:

import os
import json
import joblib
import logging
from datetime import datetime, UTC

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import FeatureHasher

import lightgbm as lgb
import optuna
import matplotlib.pyplot as plt
import platform
import sklearn as sk

# Threading/parallelism controls — keep training stable
os.environ["LOKY_MAX_CPU_COUNT"] = "1"
os.environ["JOBLIB_MULTIPROCESSING"] = "0"
os.environ["OMP_NUM_THREADS"] = "1"

# -------------------------
# Logging
# -------------------------
load_dotenv("config.env")
os.makedirs("logs", exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("logs/logistics_model.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger.info(
    "Env versions: Python %s | numpy %s | pandas %s | sklearn %s | lightgbm %s",
    platform.python_version(), np.__version__, pd.__version__, sk.__version__, lgb.__version__
)


2025-09-07 19:17:58,199 - INFO - Env versions: Python 3.13.7 | numpy 2.3.2 | pandas 2.3.1 | sklearn 1.7.1 | lightgbm 4.6.0


## Data cleaning & feature engineering helpers

In [9]:

def handle_missing_data_smart(df: pd.DataFrame) -> pd.DataFrame:
    """Carrier-aware imputation. Dropping only rows with missing target Cost."""
    df = df.copy()
    initial_count = len(df)
    df = df.dropna(subset=['Cost'])
    logger.info("Removed %d rows with missing Cost", initial_count - len(df))

    critical = ['Distance_miles', 'Weight_kg', 'origin_warehouse', 'Destination', 'Carrier']
    for c in critical:
        if c not in df.columns:
            continue
        miss = df[c].isna()
        if not miss.any():
            continue
        logger.info("Imputing %d missing values for %s", int(miss.sum()), c)
        for carrier in df['Carrier'].dropna().unique():
            m = (df['Carrier'] == carrier) & miss
            if not m.any():
                continue
            if pd.api.types.is_numeric_dtype(df[c]):
                val = df.loc[df['Carrier'] == carrier, c].median()
            else:
                mode = df.loc[df['Carrier'] == carrier, c].mode()
                val = mode.iloc[0] if len(mode) else "Unknown"
            df.loc[m, c] = val

    for c in ['origin_warehouse', 'Destination', 'Carrier']:
        if c in df.columns:
            df[c] = df[c].fillna("Unknown")
    return df

def detect_and_handle_dhl_outliers(df: pd.DataFrame) -> pd.DataFrame:
    """Cap extreme DHL costs (3*IQR above Q3)."""
    df = df.copy()
    dhl_mask = (df['Carrier'] == 'DHL')
    if not dhl_mask.any():
        return df
    dhl_df = df.loc[dhl_mask]
    Q1 = dhl_df['Cost'].quantile(0.25)
    Q3 = dhl_df['Cost'].quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 3 * IQR
    extreme = dhl_df[dhl_df['Cost'] > upper]
    if len(extreme):
        logger.info("Capping %d DHL extreme costs at %.2f", len(extreme), upper)
        df.loc[dhl_mask & (df['Cost'] > upper), 'Cost'] = upper
    return df

def analyze_dhl_data_quality(df: pd.DataFrame) -> pd.DataFrame:
    """Log data quality stats for DHL subset."""
    dhl_df = df[df['Carrier'] == 'DHL'].copy()
    logger.info("DHL samples: %d", len(dhl_df))
    for col in ['Distance_miles', 'Weight_kg', 'origin_warehouse', 'Destination']:
        if col in dhl_df.columns:
            miss = int(dhl_df[col].isna().sum())
            pct = (miss / max(len(dhl_df), 1)) if len(dhl_df) else 0.0
            logger.info("Missing %s: %d (%.1f%%)", col, miss, pct * 100)
    if len(dhl_df):
        logger.info(
            "Distance %.0f-%.0f | Weight %.1f-%.1f | Cost %.0f-%.0f",
            dhl_df['Distance_miles'].min(), dhl_df['Distance_miles'].max(),
            dhl_df['Weight_kg'].min(), dhl_df['Weight_kg'].max(),
            dhl_df['Cost'].min(), dhl_df['Cost'].max()
        )
    return dhl_df

def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    """Light, leakage-free features."""
    df = df.copy()
    if 'shipment_date' in df.columns:
        dt = pd.to_datetime(df['shipment_date'], errors='coerce')
        df['month'] = dt.dt.month.fillna(0).astype(int)
        df['day_of_week'] = dt.dt.dayofweek.fillna(0).astype(int)
        df['day_of_month'] = dt.dt.day.fillna(0).astype(int)
        df['is_weekend'] = (dt.dt.dayofweek >= 5).astype(int)
    else:
        df['month'] = 0; df['day_of_week'] = 0; df['day_of_month'] = 0; df['is_weekend'] = 0
    df['is_holiday_season'] = df['month'].isin([11, 12]).astype(int)
    df['route'] = df["origin_warehouse"].astype(str) + "→" + df["Destination"].astype(str)
    df['is_dhl'] = (df['Carrier'] == 'DHL').astype(int)
    df['is_usps'] = (df['Carrier'] == 'USPS').astype(int)
    df['is_ups'] = (df['Carrier'] == 'UPS').astype(int)
    df['is_fedex'] = (df['Carrier'] == 'FedEx').astype(int)
    df['dhl_distance'] = df['is_dhl'] * df['Distance_miles']
    df['usps_distance'] = df['is_usps'] * df['Distance_miles']
    df['dhl_weight'] = df['is_dhl'] * df['Weight_kg']
    df['usps_weight'] = df['is_usps'] * df['Weight_kg']
    df['dhl_weekend'] = ((df['Carrier'] == 'DHL') & (df['is_weekend'] == 1)).astype(int)
    df['dhl_holiday'] = ((df['Carrier'] == 'DHL') & (df['is_holiday_season'] == 1)).astype(int)
    return df

def enhanced_dhl_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dhl = (df['Carrier'] == 'DHL')
    df['dhl_distance_tier'] = 0
    if dhl.any():
        df.loc[dhl, 'dhl_distance_tier'] = pd.cut(
            df.loc[dhl, 'Distance_miles'], bins=[0,500,1000,1500,2000,2500,3000], labels=False
        ).astype('float').fillna(0).astype(int)
    df['dhl_weight_tier'] = 0
    if dhl.any():
        df.loc[dhl, 'dhl_weight_tier'] = pd.cut(
            df.loc[dhl, 'Weight_kg'], bins=[0,10,20,30,40,50,100], labels=False
        ).astype('float').fillna(0).astype(int)
    return df

def enhanced_dhl_distance_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dhl = (df['Carrier'] == 'DHL')
    df['dhl_distance_tier_detailed'] = 0
    if dhl.any():
        df.loc[dhl, 'dhl_distance_tier_detailed'] = pd.cut(
            df.loc[dhl, 'Distance_miles'], bins=[0,300,600,900,1200,1500,1800,2100,2400,3000], labels=False
        ).astype('float').fillna(0).astype(int)
    df['dhl_very_long_distance'] = (dhl & (df['Distance_miles'] > 2000)).astype(int)
    df['dhl_distance_squared'] = df['is_dhl'] * (df['Distance_miles'] ** 2)
    return df

def enhanced_dhl_weight_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    dhl = (df['Carrier'] == 'DHL')
    df['dhl_weight_tier_detailed'] = 0
    if dhl.any():
        df.loc[dhl, 'dhl_weight_tier_detailed'] = pd.cut(
            df.loc[dhl, 'Weight_kg'], bins=[0,5,10,15,20,25,30,35,40,50,100], labels=False
        ).astype('float').fillna(0).astype(int)
    df['dhl_heavy_package'] = (dhl & (df['Weight_kg'] > 30)).astype(int)
    df['dhl_very_heavy_package'] = (dhl & (df['Weight_kg'] > 35)).astype(int)
    df['dhl_weight_distance'] = df['is_dhl'] * df['Weight_kg'] * df['Distance_miles']
    return df

def add_problem_route_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    problem_routes = [
        'Warehouse_CHI→Houston',
        'Warehouse_HOU→Chicago',
        'Warehouse_MIA→San Francisco',
        'Warehouse_ATL→Denver',
        'Warehouse_NYC→Phoenix'
    ]
    df['is_problem_route'] = df['route'].isin(problem_routes).astype(int)
    df['is_long_distance_dhl'] = ((df['Carrier'] == 'DHL') & (df['Distance_miles'] > 1900)).astype(int)
    df['dhl_long_distance'] = df['is_dhl'] * (df['Distance_miles'] > 1900).astype(int)
    df['problem_route_distance'] = df['is_problem_route'] * df['Distance_miles']
    return df

def enhanced_days_features(df: pd.DataFrame) -> pd.DataFrame:
    """Placeholder to mirror earlier structure (no leakage here)."""
    return df.copy()


## Calibration, analysis & prediction helpers

In [10]:

def calculate_dynamic_calibration(df_context, y_true, y_pred, target_type='cost'):
    """Per-carrier median ratio (cost) or mean error (days)."""
    calibration = {}
    carriers = pd.Series(df_context['Carrier']).fillna("Unknown").values
    for carrier in pd.unique(carriers):
        m = (carriers == carrier)
        if m.sum() < 5:
            continue
        a = y_true[m]; p = y_pred[m]
        if target_type == 'cost':
            with np.errstate(divide='ignore', invalid='ignore'):
                ratios = a / p
                ratios = ratios[np.isfinite(ratios)]
            if len(ratios) > 0:
                calibration[carrier] = {'multiplier': float(np.median(ratios))}
        else:
            calibration[carrier] = {'additive': float(np.mean(a - p))}
    return calibration

def analyze_dhl_errors(df_val, y_true, y_pred, target_type='cost'):
    dhl_mask = (df_val['Carrier'] == 'DHL').to_numpy()
    if not dhl_mask.any():
        logger.info("No DHL samples in validation for analysis")
        return
    err = np.abs(y_true[dhl_mask] - y_pred[dhl_mask])
    logger.info("DHL %s error | mean=%.2f, median=%.2f, max=%.2f",
                target_type, err.mean(), np.median(err), err.max())

def handle_problematic_routes(predictions, df_context):
    calibrated = np.array(predictions).copy()
    route_corrections = {
        'Warehouse_CHI→Houston': 0.7,
        'Warehouse_HOU→Chicago': 0.8,
        'Warehouse_MIA→San Francisco': 0.85,
        'Warehouse_ATL→Denver': 0.9,
        'Warehouse_NYC→Phoenix': 0.9
    }
    routes = df_context['route'].astype(str).values
    for route, corr in route_corrections.items():
        m = (routes == route)
        if np.any(m):
            calibrated[m] *= corr
    m_long = ((df_context['Carrier'] == 'DHL').to_numpy() & (df_context['Distance_miles'].to_numpy() > 1900))
    if np.any(m_long):
        calibrated[m_long] *= 0.85
    return calibrated

def apply_calibrated_predictions(model, X_pre, df_context, target_type, calibration_factors):
    raw = model.predict(X_pre, num_iteration=getattr(model, "best_iteration", None))
    if target_type == 'cost':
        raw = np.expm1(raw)
    calibrated = np.array(raw).copy()
    carriers = df_context['Carrier'].fillna("Unknown").values
    if calibration_factors:
        for carrier, factors in calibration_factors.items():
            m = (carriers == carrier)
            if not np.any(m):
                continue
            if target_type == 'cost':
                mult = float(factors.get('multiplier', 1.0))
                calibrated[m] *= mult
            else:
                addv = float(factors.get('additive', 0.0))
                calibrated[m] += addv
    if target_type == 'cost':
        calibrated = handle_problematic_routes(calibrated, df_context)
    return calibrated

def predict_with_dhl_specialized_models(models, X_pre, df_meta, target_type='cost'):
    preds = np.zeros(len(df_meta))
    dhl_mask = (df_meta['Carrier'] == 'DHL').to_numpy()
    if 'dhl_long_distance' in models:
        m = dhl_mask & (df_meta['Distance_miles'].to_numpy() > 2000)
        idx = np.where(m)[0]
        if idx.size:
            preds[idx] = models['dhl_long_distance'].predict(X_pre[idx], num_iteration=getattr(models['dhl_long_distance'], "best_iteration", None))
    if 'dhl_heavy' in models:
        m = dhl_mask & (df_meta['Weight_kg'].to_numpy() > 30)
        idx = np.where(m)[0]
        if idx.size:
            preds[idx] = models['dhl_heavy'].predict(X_pre[idx], num_iteration=getattr(models['dhl_heavy'], "best_iteration", None))
    if 'dhl_normal' in models:
        m = dhl_mask & (preds == 0)
        idx = np.where(m)[0]
        if idx.size:
            preds[idx] = models['dhl_normal'].predict(X_pre[idx], num_iteration=getattr(models['dhl_normal'], "best_iteration", None))
    if target_type == 'cost':
        idx = np.where(dhl_mask)[0]
        preds[idx] = np.expm1(preds[idx])
    return preds

def predict_with_carrier_specific_models(models, X_pre, df_meta, target_type='cost'):
    preds = np.zeros(len(df_meta))
    dhl_models = {k: v for k, v in models.items() if k.startswith('dhl_')}
    if dhl_models:
        dhl_preds = predict_with_dhl_specialized_models(dhl_models, X_pre, df_meta, target_type)
        dhl_mask = (df_meta['Carrier'] == 'DHL').to_numpy()
        preds[dhl_mask] = dhl_preds[dhl_mask]
    if 'normal' in models and models['normal'] is not None:
        normal_carriers = ['UPS', 'FedEx', 'Amazon Logistics', 'LaserShip', 'OnTrac', 'USPS']
        m = (df_meta['Carrier'].isin(normal_carriers)).to_numpy() & (preds == 0)
        if np.any(m):
            yhat = models['normal'].predict(X_pre[m], num_iteration=getattr(models['normal'], "best_iteration", None))
            if target_type == 'cost':
                yhat = np.expm1(yhat)
            preds[m] = yhat
    return preds

def compute_adaptive_weights(global_preds, carrier_preds, y_cost_log, df_meta):
    """Return dict carrier -> w (weight for global); blend as w*global + (1-w)*carrier."""
    weights = {}
    carriers = df_meta['Carrier'].fillna("Unknown").values
    ytrue = np.expm1(y_cost_log.values if hasattr(y_cost_log, 'values') else y_cost_log)
    for carrier in pd.unique(carriers):
        m = (carriers == carrier)
        if m.sum() < 5:
            weights[carrier] = 0.5
            continue
        act = ytrue[m]
        g = global_preds[m]
        c = carrier_preds[m]
        rmse_g = np.sqrt(mean_squared_error(act, g))
        rmse_c = np.sqrt(mean_squared_error(act, c))
        inv_g = 0.0 if not np.isfinite(rmse_g) or rmse_g <= 0 else 1.0 / rmse_g
        inv_c = 0.0 if not np.isfinite(rmse_c) or rmse_c <= 0 else 1.0 / rmse_c
        denom = inv_g + inv_c
        w = inv_g / denom if denom > 0 else 0.5
        weights[carrier] = float(np.clip(w, 0.0, 1.0))
    return weights


## Models & transformers

In [11]:

class RouteHashingEncoder(BaseEstimator, TransformerMixin):
    """Hash a single 'route' column via sklearn's FeatureHasher.\n
    Expects a Series/array-like of strings. Produces dense array."""
    def __init__(self, n_features=256):
        self.n_features = n_features
        self.hasher = FeatureHasher(n_features=n_features, input_type='string')
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if isinstance(X, pd.DataFrame):
            strings = X.iloc[:, 0].astype(str).tolist()
        else:
            strings = pd.Series(np.asarray(X).ravel()).astype(str).tolist()
        samples = [[s] for s in strings]  # iterable of iterables of strings
        mat = self.hasher.transform(samples)
        return mat.toarray()

def make_ohe():
    """Sklearn-agnostic OneHotEncoder (sparse_output for >=1.4; fallback to sparse)."""
    try:
        return OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown='ignore', sparse=False)

def cv_lightgbm(X, y, params, scoring, cv=5):
    scores = []
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    for tr, va in kf.split(X):
        Xtr, Xva = X[tr], X[va]
        ytr, yva = y[tr], y[va]
        dtr = lgb.Dataset(Xtr, label=ytr)
        dva = lgb.Dataset(Xva, label=yva)
        model = lgb.train(
            params, dtr, valid_sets=[dva],
            num_boost_round=1000,
            callbacks=[lgb.early_stopping(stopping_rounds=50)]
        )
        if scoring == 'rmse_expm1':
            yhat = np.expm1(model.predict(Xva, num_iteration=getattr(model, "best_iteration", None)))
            score = np.sqrt(mean_squared_error(np.expm1(yva), yhat))
        else:
            yhat = model.predict(Xva, num_iteration=getattr(model, "best_iteration", None))
            score = np.sqrt(mean_squared_error(yva, yhat))
        scores.append(score)
    return np.array(scores)

def plot_carrier_rmse(carrier_rmse: pd.DataFrame, output_dir: str):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    df1 = carrier_rmse.sort_values('RMSE_Cost_Calibrated')
    ax1.bar(df1['Carrier'], df1['RMSE_Cost_Calibrated'], alpha=0.7)
    ax1.set_title('Cost RMSE (Calibrated) by Carrier')
    ax1.set_ylabel('RMSE ($)')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(axis='y', alpha=0.3)
    df2 = carrier_rmse.sort_values('RMSE_Transit_Days')
    ax2.bar(df2['Carrier'], df2['RMSE_Transit_Days'], alpha=0.7)
    ax2.set_title('Transit Days RMSE by Carrier')
    ax2.set_ylabel('RMSE (Days)')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(os.path.join(output_dir, "carrier_rmse_analysis.png"), dpi=300, bbox_inches="tight")
    plt.close()

def train_advanced_dhl_model(X_train, y_train):
    if len(X_train) < 30:
        logger.warning("Insufficient DHL data for advanced training")
        return None
    params = {
        'objective': 'regression', 'metric': 'rmse',
        'learning_rate': 0.005, 'num_leaves': 127, 'max_depth': 12,
        'min_child_samples': 5, 'feature_fraction': 0.6,
        'bagging_fraction': 0.6, 'bagging_freq': 5,
        'lambda_l1': 0.5, 'lambda_l2': 0.5, 'min_data_in_leaf': 10, 'verbose': -1
    }
    dtrain = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(params, dtrain, num_boost_round=1000)
    return model

def train_dhl_specialized_models(X_train, y_train, df_train):
    models = {}
    y_np = y_train.values if hasattr(y_train, 'values') else y_train
    dhl = (df_train['Carrier'] == 'DHL').to_numpy()
    long_m = dhl & (df_train['Distance_miles'].to_numpy() > 2000)
    if long_m.sum() > 10:
        idx = np.where(long_m)[0]
        dtr = lgb.Dataset(X_train[idx], label=y_np[idx])
        models['dhl_long_distance'] = lgb.train({
            'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.002,
            'num_leaves': 15, 'max_depth': 4, 'min_child_samples': 5, 'feature_fraction': 0.8,
            'lambda_l1': 1.0, 'lambda_l2': 1.0, 'verbose': -1
        }, dtr, num_boost_round=500)
    heavy_m = dhl & (df_train['Weight_kg'].to_numpy() > 30)
    if heavy_m.sum() > 5:
        idx = np.where(heavy_m)[0]
        dtr = lgb.Dataset(X_train[idx], label=y_np[idx])
        models['dhl_heavy'] = lgb.train({
            'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.005,
            'num_leaves': 20, 'max_depth': 5, 'min_child_samples': 3, 'feature_fraction': 0.7,
            'lambda_l1': 0.8, 'lambda_l2': 0.8, 'verbose': -1
        }, dtr, num_boost_round=300)
    normal_m = dhl & (~long_m) & (~heavy_m)
    if normal_m.sum() > 20:
        idx = np.where(normal_m)[0]
        models['dhl_normal'] = train_advanced_dhl_model(X_train[idx], y_np[idx])
    return models

def train_carrier_specific_models(X_train, y_train, X_val, y_val, df_train, df_val, target_type='cost'):
    models = {}
    if target_type == 'cost':
        models.update(train_dhl_specialized_models(X_train, y_train, df_train))
    normal_carriers = ['UPS', 'FedEx', 'Amazon Logistics', 'LaserShip', 'OnTrac', 'USPS']
    norm = df_train['Carrier'].isin(normal_carriers).to_numpy()
    if norm.any():
        Xn = X_train[norm]
        yn = (y_train.values if hasattr(y_train, 'values') else y_train)[norm]
        vn = df_val['Carrier'].isin(normal_carriers).to_numpy()
        Xvn = X_val[vn]
        yvn = (y_val.values if hasattr(y_val, 'values') else y_val)[vn]
        params = {
            'objective': 'regression', 'metric': 'rmse', 'learning_rate': 0.05, 'num_leaves': 31,
            'max_depth': 8, 'min_child_samples': 10, 'feature_fraction': 0.8, 'verbose': -1
        }
        dtr = lgb.Dataset(Xn, label=yn)
        if len(Xvn) > 0:
            dva = lgb.Dataset(Xvn, label=yvn)
            models['normal'] = lgb.train(params, dtr, valid_sets=[dva],
                                         num_boost_round=1000,
                                         callbacks=[lgb.early_stopping(stopping_rounds=30)])
        else:
            models['normal'] = lgb.train(params, dtr, num_boost_round=1000)
    return models


## Data loading (MySQL with CSV fallback)

In [12]:

def load_dataset() -> pd.DataFrame:
    """Try MySQL first; fallback to CSV via LOGISTICS_CSV_PATH; else raise."""
    # MySQL
    try:
        engine = create_engine(
            f"mysql+mysqlconnector://{os.getenv('MYSQL_USER','root')}:{os.getenv('MYSQL_PASSWORD','password')}"
            f"@{os.getenv('MYSQL_HOST','localhost')}:{os.getenv('MYSQL_PORT','3306')}/{os.getenv('MYSQL_DB','sys')}"
        )
        df = pd.read_sql("SELECT * FROM us_logistics;", engine)
        if not df.empty:
            logger.info("Loaded %s rows from MySQL us_logistics", len(df))
            return df
    except Exception as e:
        logger.warning("MySQL read failed: %s", e)
    # CSV
    path = os.getenv("LOGISTICS_CSV_PATH", "logistics_shipments_dataset.csv")
    if os.path.exists(path):
        df = pd.read_csv(path)
        logger.info("Loaded %s rows from CSV %s", len(df), path)
        return df
    raise RuntimeError("Could not load data from MySQL or CSV. Set DB env vars or LOGISTICS_CSV_PATH.")


## Main training pipeline

In [13]:

def main():
    try:
        logger.info("=== START TRAINING PIPELINE ===")
        df = load_dataset()

        # Basic standardization
        df = handle_missing_data_smart(df)
        # Global IQR trim for Cost
        if "Cost" in df.columns:
            Q1 = df["Cost"].quantile(0.25); Q3 = df["Cost"].quantile(0.75); IQR = Q3 - Q1
            lb, ub = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
            before = len(df)
            df = df[(df["Cost"] >= lb) & (df["Cost"] <= ub)]
            logger.info("Removed %d global cost outliers", before - len(df))

        df = detect_and_handle_dhl_outliers(df)
        df = feature_engineering(df)
        df = enhanced_dhl_features(df)
        df = enhanced_dhl_distance_features(df)
        df = enhanced_dhl_weight_features(df)
        df = add_problem_route_features(df)
        df = enhanced_days_features(df)

        df['Cost_log'] = np.log1p(df['Cost'])

        # Split
        carrier_counts = df['Carrier'].value_counts()
        stratify_col = df['Carrier'] if (len(carrier_counts) > 1 and carrier_counts.min() > 1) else None

        base_num_cols = [
            'Distance_miles','Weight_kg','month','day_of_week','day_of_month',
            'is_weekend','is_holiday_season',
            'is_dhl','is_usps','is_ups','is_fedex',
            'dhl_distance','usps_distance','dhl_weight','usps_weight',
            'is_problem_route','is_long_distance_dhl','dhl_long_distance','problem_route_distance',
            'dhl_distance_tier','dhl_weight_tier','dhl_weekend','dhl_holiday',
            'dhl_distance_tier_detailed','dhl_very_long_distance','dhl_distance_squared',
            'dhl_weight_tier_detailed','dhl_heavy_package','dhl_very_heavy_package','dhl_weight_distance'
        ]
        base_num_cols = [c for c in base_num_cols if c in df.columns]
        cat_cols_no_route = [c for c in ['Carrier','origin_warehouse','Destination'] if c in df.columns]

        cols_for_split = ['Carrier','origin_warehouse','Destination','route'] + base_num_cols + ['Cost_log']
        if 'Transit_Days' in df.columns: cols_for_split.append('Transit_Days')
        cols_for_split = list(dict.fromkeys([c for c in cols_for_split if c in df.columns]))

        df_split = df[cols_for_split].copy()
        y_cost = df_split['Cost_log']
        y_days = df_split['Transit_Days'] if 'Transit_Days' in df_split.columns else pd.Series(0, index=df_split.index)

        X_train, X_val, y_cost_train, y_cost_val, y_days_train, y_days_val = train_test_split(
            df_split.drop(columns=['Cost_log','Transit_Days'], errors='ignore'),
            y_cost, y_days, test_size=0.2, random_state=42, stratify=stratify_col
        )
        df_train = df.loc[X_train.index].copy()
        df_val = df.loc[X_val.index].copy()

        # Post-split aggregates (train-only)
        cr_counts = df_train.groupby(['Carrier','route'], observed=False).size().rename('carrier_route_count')
        def add_train_only_counts(dframe):
            idx = dframe.set_index(['Carrier','route']).index
            vals = idx.map(cr_counts).astype('float').fillna(0.0).values
            return vals
        X_train['carrier_route_count'] = add_train_only_counts(X_train)
        X_val['carrier_route_count'] = add_train_only_counts(X_val)

        def add_days_aggregates(d_target):
            if 'Transit_Days' not in df_train.columns:
                for c in ['carrier_avg_days','route_avg_days','carrier_route_avg_days']:
                    d_target[c] = 0.0
                return d_target
            carrier_map = df_train.groupby('Carrier', observed=False)['Transit_Days'].mean()
            route_map = df_train.groupby('route', observed=False)['Transit_Days'].mean()
            cr_map = df_train.groupby(['Carrier','route'], observed=False)['Transit_Days'].mean()
            gl = df_train['Transit_Days'].mean()
            d_target['carrier_avg_days'] = d_target['Carrier'].map(carrier_map)
            d_target['route_avg_days'] = d_target['route'].map(route_map)
            d_target['carrier_route_avg_days'] = d_target.set_index(['Carrier','route']).index.map(cr_map)
            for c in ['carrier_avg_days','route_avg_days','carrier_route_avg_days']:
                d_target[c] = d_target[c].astype('float').fillna(gl)
            return d_target
        X_train_days = add_days_aggregates(X_train.copy())
        X_val_days = add_days_aggregates(X_val.copy())

        analyze_dhl_data_quality(df)

        # Preprocessors
        num_cols_cost = base_num_cols + ['carrier_route_count']
        cat_cols_cost = cat_cols_no_route
        num_cols_days = base_num_cols + ['carrier_route_count','carrier_avg_days','route_avg_days','carrier_route_avg_days']
        cat_cols_days = cat_cols_no_route

        num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])
        cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')), ('onehot', make_ohe())])

        preprocessor_cost = ColumnTransformer(
            transformers=[
                ('num', num_pipe, num_cols_cost),
                ('cat', cat_pipe, cat_cols_cost),
                ('route_hash', RouteHashingEncoder(n_features=256), ['route'])
            ], remainder='drop'
        )
        preprocessor_days = ColumnTransformer(
            transformers=[
                ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_cols_days),
                ('cat', Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='Unknown')), ('onehot', make_ohe())]), cat_cols_days),
                ('route_hash', RouteHashingEncoder(n_features=256), ['route'])
            ], remainder='drop'
        )

        # Dtypes
        for dfX in (X_train, X_val, X_train_days, X_val_days):
            for c in ['Distance_miles','Weight_kg']:
                if c in dfX.columns: dfX[c] = dfX[c].astype('float')
            for c in cat_cols_no_route + ['route']:
                if c in dfX.columns: dfX[c] = dfX[c].fillna("Unknown").astype(str)

        # Fit/transform
        X_train_pre_cost = preprocessor_cost.fit_transform(X_train)
        X_val_pre_cost = preprocessor_cost.transform(X_val)
        X_train_pre_days = preprocessor_days.fit_transform(X_train_days)
        X_val_pre_days = preprocessor_days.transform(X_val_days)

        # Numpy
        Xtr_c = X_train_pre_cost if isinstance(X_train_pre_cost, np.ndarray) else X_train_pre_cost.toarray() if hasattr(X_train_pre_cost, "toarray") else X_train_pre_cost
        Xva_c = X_val_pre_cost if isinstance(X_val_pre_cost, np.ndarray) else X_val_pre_cost.toarray() if hasattr(X_val_pre_cost, "toarray") else X_val_pre_cost
        ytr_c = y_cost_train.values if hasattr(y_cost_train,'values') else y_cost_train
        yva_c = y_cost_val.values if hasattr(y_cost_val,'values') else y_cost_val

        Xtr_d = X_train_pre_days if isinstance(X_train_pre_days, np.ndarray) else X_train_pre_days.toarray() if hasattr(X_train_pre_days, "toarray") else X_train_pre_days
        Xva_d = X_val_pre_days if isinstance(X_val_pre_days, np.ndarray) else X_val_pre_days.toarray() if hasattr(X_val_pre_days, "toarray") else X_val_pre_days
        ytr_d = y_days_train.values if hasattr(y_days_train,'values') else y_days_train
        yva_d = y_days_val.values if hasattr(y_days_val,'values') else y_days_val

        # Optuna tuning
        def get_common_params(trial):
            return {
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 200),
                'max_depth': trial.suggest_int('max_depth', 5, 20),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
                'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
                'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
            }
        def objective_cost(trial):
            params = get_common_params(trial)
            params.update({'objective':'regression','metric':'rmse','verbose':-1,'random_state':42})
            dtr = lgb.Dataset(Xtr_c, label=ytr_c); dva = lgb.Dataset(Xva_c, label=yva_c)
            mdl = lgb.train(params, dtr, valid_sets=[dva], num_boost_round=1000,
                            callbacks=[lgb.log_evaluation(0), lgb.early_stopping(stopping_rounds=50)])
            yhat = np.expm1(mdl.predict(Xva_c, num_iteration=getattr(mdl,"best_iteration",None)))
            rmse = np.sqrt(mean_squared_error(np.expm1(yva_c), yhat))
            return rmse
        def objective_days(trial):
            params = get_common_params(trial)
            params.update({'objective':'poisson','metric':'rmse','verbose':-1,'random_state':42})
            dtr = lgb.Dataset(Xtr_d, label=ytr_d); dva = lgb.Dataset(Xva_d, label=yva_d)
            mdl = lgb.train(params, dtr, valid_sets=[dva], num_boost_round=1000,
                            callbacks=[lgb.log_evaluation(0), lgb.early_stopping(stopping_rounds=50)])
            yhat = mdl.predict(Xva_d, num_iteration=getattr(mdl,"best_iteration",None))
            rmse = np.sqrt(mean_squared_error(yva_d, yhat))
            return rmse

        logger.info("Tuning with Optuna")
        study_cost = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
        study_cost.optimize(objective_cost, n_trials=30, show_progress_bar=False)
        study_days = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
        study_days.optimize(objective_days, n_trials=30, show_progress_bar=False)
        logger.info("Best RMSE (cost)=%.4f | (days)=%.4f", study_cost.best_value, study_days.best_value)

        best_params_cost = {**study_cost.best_params, 'objective':'regression','metric':'rmse','verbose':-1,'random_state':42}
        best_params_days = {**study_days.best_params, 'objective':'poisson','metric':'rmse','verbose':-1,'random_state':42}

        # Final global models
        mdl_cost = lgb.train(best_params_cost, lgb.Dataset(Xtr_c, label=ytr_c),
                             valid_sets=[lgb.Dataset(Xva_c, label=yva_c)],
                             num_boost_round=1000, callbacks=[lgb.log_evaluation(100), lgb.early_stopping(stopping_rounds=50)])
        mdl_days = lgb.train(best_params_days, lgb.Dataset(Xtr_d, label=ytr_d),
                             valid_sets=[lgb.Dataset(Xva_d, label=yva_d)],
                             num_boost_round=1000, callbacks=[lgb.log_evaluation(100), lgb.early_stopping(stopping_rounds=50)])

        # Carrier-specific cost models
        logger.info("Training carrier-specific models")
        cost_models = train_carrier_specific_models(Xtr_c, ytr_c, Xva_c, yva_c, df_train, df_val, 'cost')

        # Calibration
        logger.info("Calculating calibration factors")
        val_cost_raw = mdl_cost.predict(Xva_c, num_iteration=getattr(mdl_cost,"best_iteration",None))
        val_cost_exp = np.expm1(val_cost_raw)
        cost_cal = calculate_dynamic_calibration(df_val, np.expm1(yva_c), val_cost_exp, 'cost')
        val_days = mdl_days.predict(Xva_d, num_iteration=getattr(mdl_days,"best_iteration",None))
        days_cal = calculate_dynamic_calibration(df_val, yva_d, val_days, 'days')

        # Error analysis
        analyze_dhl_errors(df_val, np.expm1(yva_c), val_cost_exp, 'cost')

        # Adaptive weights (for eval/reporting)
        global_val = np.expm1(mdl_cost.predict(Xva_c, num_iteration=getattr(mdl_cost,"best_iteration",None)))
        carrier_val = predict_with_carrier_specific_models(cost_models, Xva_c, df_val, 'cost')
        carrier_val = np.where(carrier_val == 0, global_val, carrier_val)
        adaptive_w = compute_adaptive_weights(global_val, carrier_val, y_cost_val, df_val)

        # CV
        cv_cost = cv_lightgbm(Xtr_c, ytr_c, best_params_cost, 'rmse_expm1', cv=5)
        cv_days = cv_lightgbm(Xtr_d, ytr_d, best_params_days, 'rmse', cv=5)
        logger.info("CV RMSE Cost: %.4f ± %.4f | Days: %.4f ± %.4f", cv_cost.mean(), cv_cost.std(), cv_days.mean(), cv_days.std())

        # Per-carrier evaluation
        def evaluate_per_carrier(Xc, Xd, ylog, ydays, meta_df, mdl_c, mdl_d, group_models, cal_cost, cal_days, w_adapt):
            carriers = sorted(meta_df['Carrier'].dropna().unique())
            cost_global = apply_calibrated_predictions(mdl_c, Xc, meta_df, 'cost', cal_cost)
            days_global = apply_calibrated_predictions(mdl_d, Xd, meta_df, 'days', cal_days)
            # adaptive blend
            cost_adapt = np.zeros(len(meta_df))
            gpred = np.expm1(mdl_c.predict(Xc, num_iteration=getattr(mdl_c,"best_iteration",None)))
            cpred = predict_with_carrier_specific_models(group_models, Xc, meta_df, 'cost')
            for i, car in enumerate(meta_df['Carrier'].values):
                if car == 'DHL' and any(k.startswith('dhl_') for k in group_models.keys()):
                    cost_adapt[i] = cpred[i]
                else:
                    w = w_adapt.get(car, 0.5)
                    c = cpred[i] if cpred[i] != 0 else gpred[i]
                    cost_adapt[i] = w * gpred[i] + (1 - w) * c
            rows = []
            for car in carriers:
                m = (meta_df['Carrier'].values == car)
                if not np.any(m): continue
                rmse_cg = float(np.sqrt(mean_squared_error(np.expm1(ylog.values[m]), cost_global[m])))
                rmse_ca = float(np.sqrt(mean_squared_error(np.expm1(ylog.values[m]), cost_adapt[m])))
                rmse_d  = float(np.sqrt(mean_squared_error(ydays.values[m], days_global[m])))
                rows.append({"Carrier":car,"Samples":int(m.sum()),
                             "RMSE_Cost_Calibrated":rmse_cg,
                             "RMSE_Cost_Adaptive":rmse_ca,
                             "RMSE_Transit_Days":rmse_d})
            return pd.DataFrame(rows)

        carrier_rmse = evaluate_per_carrier(
            Xva_c, Xva_d, y_cost_val, y_days_val, df_val,
            mdl_cost, mdl_days, cost_models, cost_cal, days_cal, adaptive_w
        )

        # Save artifacts
        out = "model_artifacts"; os.makedirs(out, exist_ok=True)
        joblib.dump(mdl_cost, os.path.join(out, "cost_model.pkl"))
        joblib.dump(mdl_days, os.path.join(out, "transit_days_model.pkl"))
        joblib.dump(preprocessor_cost, os.path.join(out, "preprocessor_cost.pkl"))
        joblib.dump(preprocessor_days, os.path.join(out, "preprocessor_days.pkl"))
        joblib.dump(cost_models, os.path.join(out, "cost_models_grouped.pkl"))
        # Generic names for Streamlit sidebar
        joblib.dump(preprocessor_cost, os.path.join(out, "preprocessor.pkl"))
        joblib.dump(mdl_cost, os.path.join(out, "cost_model.pkl"))

        # best_params.json (features + calibration + metrics)
        best = {
            "cost_params": best_params_cost,
            "days_params": best_params_days,
            "cost_features": {"numerical": list(num_cols_cost), "categorical": list(cat_cols_cost), "route_hash_features": 256},
            "days_features": {"numerical": list(num_cols_days), "categorical": list(cat_cols_days), "route_hash_features": 256},
            "cv_scores": {"cost_mean": float(cv_cost.mean()), "cost_std": float(cv_cost.std()),
                          "days_mean": float(cv_days.mean()), "days_std": float(cv_days.std())},
            "calibration": {"cost": cost_cal, "days": days_cal}
        }
        # validation RMSE summary
        cost_val_cal = apply_calibrated_predictions(mdl_cost, Xva_c, df_val, 'cost', cost_cal)
        cost_val_adapt = np.zeros(len(df_val))
        gpred = np.expm1(mdl_cost.predict(Xva_c, num_iteration=getattr(mdl_cost,"best_iteration",None)))
        cpred = predict_with_carrier_specific_models(cost_models, Xva_c, df_val, 'cost')
        for i, car in enumerate(df_val['Carrier'].values):
            if car == 'DHL' and any(k.startswith('dhl_') for k in cost_models.keys()):
                cost_val_adapt[i] = cpred[i]
            else:
                w = adaptive_w.get(car, 0.5)
                c = cpred[i] if cpred[i] != 0 else gpred[i]
                cost_val_adapt[i] = w * gpred[i] + (1 - w) * c
        rmse_cost_cal = float(np.sqrt(mean_squared_error(np.expm1(yva_c), cost_val_cal)))
        rmse_cost_adp = float(np.sqrt(mean_squared_error(np.expm1(yva_c), cost_val_adapt)))
        days_val_cal = apply_calibrated_predictions(mdl_days, Xva_d, df_val, 'days', days_cal)
        rmse_days_cal = float(np.sqrt(mean_squared_error(yva_d, days_val_cal)))
        best["validation_rmse"] = {"cost_calibrated": rmse_cost_cal, "cost_adaptive": rmse_cost_adp, "days_calibrated": rmse_days_cal}

        with open(os.path.join(out, "best_params.json"), "w") as f:
            json.dump(best, f, indent=2)

        carrier_rmse.to_csv(os.path.join(out, "carrier_performance.csv"), index=False)
        plot_carrier_rmse(carrier_rmse, out)

        logger.info("Artifacts saved to %s", out)
        logger.info("=== TRAINING COMPLETED SUCCESSFULLY ===")

        return {
            "artifacts_dir": out,
            "best_params_json": os.path.join(out, "best_params.json"),
            "carrier_performance_csv": os.path.join(out, "carrier_performance.csv"),
            "rmse_plot_png": os.path.join(out, "carrier_rmse_analysis.png"),
            "when": datetime.now(UTC).isoformat().replace("+00:00", "Z")
        }

    except Exception as e:
        logger.exception("Error during pipeline execution")
        raise


## Run

In [14]:

if __name__ == "__main__":
    # Execute the pipeline when running the notebook programmatically
    summary = main()
    print("Summary:", json.dumps(summary, indent=2))


2025-09-07 19:18:06,290 - INFO - === START TRAINING PIPELINE ===
2025-09-07 19:18:06,328 - INFO - Loaded 2000 rows from MySQL us_logistics
2025-09-07 19:18:06,331 - INFO - Removed 41 rows with missing Cost
2025-09-07 19:18:06,335 - INFO - Removed 4 global cost outliers
2025-09-07 19:18:06,361 - INFO - DHL samples: 275
2025-09-07 19:18:06,362 - INFO - Missing Distance_miles: 0 (0.0%)
2025-09-07 19:18:06,362 - INFO - Missing Weight_kg: 0 (0.0%)
2025-09-07 19:18:06,362 - INFO - Missing origin_warehouse: 0 (0.0%)
2025-09-07 19:18:06,363 - INFO - Missing Destination: 0 (0.0%)
2025-09-07 19:18:06,363 - INFO - Distance 105-2499 | Weight 3.1-88.7 | Cost 32-464
2025-09-07 19:18:06,388 - INFO - Tuning with Optuna
[I 2025-09-07 19:18:06,388] A new study created in memory with name: no-name-aefcb138-dfd1-49b8-ba1b-fdfbbc03da8b
[I 2025-09-07 19:18:06,506] Trial 0 finished with value: 25.0287761646713 and parameters: {'learning_rate': 0.030710573677773714, 'num_leaves': 192, 'max_depth': 16, 'min_ch

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[357]	valid_0's rmse: 0.153029
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[116]	valid_0's rmse: 0.152512
Training until validation scores don't improve for 50 rounds


[I 2025-09-07 19:18:06,767] Trial 2 finished with value: 27.980994137976317 and parameters: {'learning_rate': 0.03647316284911211, 'num_leaves': 72, 'max_depth': 14, 'min_child_samples': 11, 'feature_fraction': 0.7168578594140872, 'bagging_fraction': 0.7465447373174766, 'bagging_freq': 5, 'lambda_l1': 0.11656915613247415, 'lambda_l2': 6.267062696005991e-07}. Best is trial 1 with value: 24.65484374287327.
[I 2025-09-07 19:18:06,847] Trial 3 finished with value: 24.287135410013047 and parameters: {'learning_rate': 0.04666963767236924, 'num_leaves': 127, 'max_depth': 5, 'min_child_samples': 32, 'feature_fraction': 0.6682096494749166, 'bagging_fraction': 0.6260206371941118, 'bagging_freq': 10, 'lambda_l1': 4.905556676028766, 'lambda_l2': 0.1886149587855396}. Best is trial 3 with value: 24.287135410013047.


Early stopping, best iteration is:
[212]	valid_0's rmse: 0.161937
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[636]	valid_0's rmse: 0.155349
Training until validation scores don't improve for 50 rounds


[I 2025-09-07 19:18:06,988] Trial 4 finished with value: 24.36299639937509 and parameters: {'learning_rate': 0.02490643969382439, 'num_leaves': 37, 'max_depth': 15, 'min_child_samples': 25, 'feature_fraction': 0.6488152939379115, 'bagging_fraction': 0.798070764044508, 'bagging_freq': 1, 'lambda_l1': 1.5271567592511939, 'lambda_l2': 2.133142332373e-06}. Best is trial 3 with value: 24.287135410013047.
[I 2025-09-07 19:18:07,039] Trial 5 finished with value: 23.611632998211867 and parameters: {'learning_rate': 0.07277150634170934, 'num_leaves': 76, 'max_depth': 13, 'min_child_samples': 30, 'feature_fraction': 0.6739417822102108, 'bagging_fraction': 0.9878338511058234, 'bagging_freq': 8, 'lambda_l1': 2.8542399074977594, 'lambda_l2': 1.1309571585271492}. Best is trial 5 with value: 23.611632998211867.
[I 2025-09-07 19:18:07,123] Trial 6 finished with value: 25.567581223641096 and parameters: {'learning_rate': 0.059963338824126605, 'num_leaves': 186, 'max_depth': 6, 'min_child_samples': 14, 

...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[456]	valid_0's rmse: 0.169446
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[534]	valid_0's rmse: 0.160388
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[387]	valid_0's rmse: 0.0895428
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[618]	valid_0's rmse: 0.0874412
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[612]	valid_0's rmse: 0.158849


2025-09-07 19:18:15,899 - INFO - CV RMSE Cost: 22.5275 ± 3.7490 | Days: 1.0155 ± 0.0572


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[67]	valid_0's rmse: 1.06364
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[64]	valid_0's rmse: 0.933507
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[57]	valid_0's rmse: 0.960408
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[44]	valid_0's rmse: 1.07249
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[45]	valid_0's rmse: 1.0477


2025-09-07 19:18:16,217 - INFO - Artifacts saved to model_artifacts
2025-09-07 19:18:16,217 - INFO - === TRAINING COMPLETED SUCCESSFULLY ===


Summary: {
  "artifacts_dir": "model_artifacts",
  "best_params_json": "model_artifacts/best_params.json",
  "carrier_performance_csv": "model_artifacts/carrier_performance.csv",
  "rmse_plot_png": "model_artifacts/carrier_rmse_analysis.png",
  "when": "2025-09-08T02:18:16.218088Z"
}
